# Arte Valiosa 

**Contexto**: É dada uma sala de museu de dimensões :
$M \times N ,$ onde existe uma porta em $(0, 0)$ e um quadro valioso em $(M, N)$. Foram instalados $K$ detectores de movimentos em posições $(x_i, y_i)$ dadas, cada um tendo um raio de ação igual a $s_i$. Um ladrão quer roubar o quadro valioso. Conseguirá fazer isso sem ser detectado?

**Entrada**: Um único caso de teste descrito em $K+1$ linhas. Na primeira linha vêm os inteiros $M, N, K$ $(10 \leq M, N \leq 10^4, 1 \leq K \leq 10^3)$. Em seguida $K$ linhas com 3 inteiros, descrevendo sua posição $x_i, y_i$ e seu raio de ação $s_i$ $(0 < x_i < M, 0 < y_i < N, 0 < s_i \leq 10^4)$.

**Saída**: Imprimir *S* se for possível o roubo sem detecção ou *N*, caso contrário.


| | |
|-|-|
|*Exemplo de entrada:* |*Exemplo de saída:*|
|10 22 2               |S   |              
|4 6 5                 |
|6 16 5                |

In [2]:
def BFS_jungle(graph):
    jungle = []   #lista que armazena as arvores geradas
    visited = set()  #conj. para armazenar os vertices ja visitados
    
    for initial_vertice in graph:
        if initial_vertice not in visited:
            queue = [initial_vertice]  #fila de vertices a serem explorados
            tree = [] #arvore gerada conectada

            while queue:
                node = queue.pop(0) # remove o primeiro vertice da fila
                if node not in visited:
                    visited.add(node) #marcar o vertice como visitado
                    tree.append(node) #adicionar vertice à arvore
                    # adicionar todos os vizinhos não visitados à fila
                    queue.extend([adj_node for adj_node in graph[node] if adj_node not in visited])
            
            jungle.append(tree)
    
    return jungle


In [3]:

#EXEMPLO 
grafo = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E'],
    'G': ['H','I'],
    'H': [],
    'J': ["I"],
    'I': ['J','G']
}

print(BFS_jungle(grafo))

[['A', 'B', 'C', 'D', 'E', 'F'], ['G', 'H', 'I', 'J']]


## Resolvendo o problema

In [26]:
def menu():
    # definindo M, N, K
    largura, comprimento, qtd_detectores = map(int, input("Digite a largura, comprimento e quantidade de detectores: ").split())

    # corrigindo as medidas considerando o inicio em 0
    largura -= 1
    comprimento -= 1
    
    sensores = {}
    # K inputs associados à posição (x,y) de cada detector e seu raio de atuação
    for num_sensor in range(qtd_detectores):
        pos_X, pos_y, area = map(int, input(f"Digite a posição horizontal, posição vertical e área do sensor {num_sensor}: ").split())
        sensores.update({f"sensor {str(num_sensor)}":[pos_X,pos_y,area]})  #adicionando sensor ao dicionario
    
    return [largura, comprimento, sensores]

def check_touch_in_wall(room_width, room_length, sensor):
    walls_touched = []
    # sensor = [pos_x, pos_y, area]
    # Verifica se o sensor toca a parede da esquerda (x = 0)
    if sensor[0] - sensor[2] <= 0:
        walls_touched.append('leftWall')

    # Verifica se o sensor toca a parede da direita (x = largura)
    if sensor[0] + sensor[2] >= room_width:
        walls_touched.append('rightWall')

    # Verifica se o sensor toca a parede de cima (y = 0)
    if sensor[1] - sensor[2] <= 0:
        walls_touched.append('topWall')

    # Verifica se o sensor toca a parede de baixo (y = comprimento)
    if sensor[1] + sensor[2] >= room_length:
        walls_touched.append('downWall')

    return walls_touched

def is_edge(sensor1, sensor2):
    # Extrai as posições e os raios dos sensores
    x1, y1, raio1 = sensor1
    x2, y2, raio2 = sensor2
    
    # Calcula a distância euclidiana entre os centros dos dois sensores
    distancia = (x2 - x1)**2 + (y2 - y1)**2
    # OBS.: preferivel utilizar tudo elevado ao quadrado = evitar problemas de precisão
    
    # Compara a distância com o quadrado da soma dos raios
    return distancia <= (raio1 + raio2)**2

def build_graph(width, length, sensors):
    graph = { 'leftWall': [], 'rightWall': [], 'topWall': [], 'downWall': [] }

    # Adiciona os sensores ao grafo e verifica se tocam as paredes
    for sensor_name, sensor_info in sensors.items():
        graph[sensor_name] = []
        walls_touched = check_touch_in_wall(width, length, sensor_info)
        for wall in walls_touched:
            # adicionando ida e volta => grafo nao direcional
            graph[sensor_name].append(wall)
            graph[wall].append(sensor_name)

    # Verifica se os sensores se interceptam e adiciona arestas entre eles
    sensor_names = list(sensors.keys())
    for i in range(len(sensor_names)):
        for j in range(i+1, len(sensor_names)):
            if is_edge(sensors[sensor_names[i]], sensors[sensor_names[j]]):
                # adicionando ida e volta => grafo nao direcional
                graph[sensor_names[i]].append(sensor_names[j])
                graph[sensor_names[j]].append(sensor_names[i])

    return graph

def thief_success(graph):
    ''' No problema, o ladrão está na posição (0,0) e o artefato em MxN. Ou seja,
    o ladrão só consiguirá caso a árvore não se conecata nos casos: 
        - leftWall e downWall
        - topWall e downWall
        - leftWall e rightWall
        - topWall e rightWall.
    Assim, verifica-se nas árvores se ocorre algum desses casos. '''

    # Define os vértices de início e fim
    start_vertex = ['leftWall', 'topWall']
    end_vertex = ['rightWall', 'downWall']

    # Realiza a busca em largura (BFS) no grafo
    bfs_result = BFS_jungle(graph)
    #print(bfs_result)
    # Verifica se existe um caminho do início ao fim sem passar pelos sensores
    for tree in bfs_result:
        if (start_vertex[0] in tree or start_vertex[1] in tree) and (end_vertex[0] in tree or end_vertex[1] in tree):
            return False  # O ladrão não consegue alcançar o artefato sem ser detectado
    
    return True  # O ladrão consegue alcançar o artefato sem ser detectado


def resolution():
    menu_options = menu()
    grafo = build_graph(menu_options[0], menu_options[1], menu_options[2])
    return print("S" if thief_success(grafo) else "N")

resolution()

N
